In [22]:
import cv2
import numpy as np

In [23]:
img1 = cv2.imread('./img/taekwonv1.jpg')
img2 = cv2.imread('./img/figures2.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [24]:
# ORB, BF-Hamming로 knnMatch
detector = cv2.ORB_create()
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

In [25]:
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(desc1, desc2)

In [26]:
# 매칭 결과를 거리 기준 오름차순 정렬
matches = sorted(matches, key=lambda x:x.distance)

In [27]:
# 모든 매칭점 그리기
res1 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None,
                      flags = cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

In [28]:
# 매칭점으로 원근 변환 및 영역 표시
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches])
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches])

In [29]:
# RANSAC으로 변환행렬 근사 계산
mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
h,w = img1.shape[:2]
pts = np.float32([ [[0, 0]], [[0, h-1]], [[w-1, h-1]], [[w-1, 0]] ])
dst = cv2.perspectiveTransform(pts, mtrx)
img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

In [30]:
# 정상치 매칭만 그리기
matchesMask = mask.ravel().tolist()
res2 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None,
                     matchesMask = matchesMask,
                     flags = cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

In [31]:
# 모든 매칭점과 정상치 비율
accuracy = float(mask.sum()) / mask.size
print(f'accuracy: {mask.sum():d}/{mask.size:d}({accuracy:.2f})')

accuracy: 25/135(0.19)


In [32]:
# 결과 출력
cv2.imshow('Matching-All', res1)
cv2.imshow('Matching-Inlier', res2)
cv2.waitKey()
cv2.destroyAllWindows()